In [1]:
using ITensors
using ITensorChemistry

In [2]:
function create_molecule(m::String, r::Float64, rep::Int64)
    s = []
    for j in 0:rep-1
        push!(s,Atom(m, j*r , 0.0, 0.0))
    end
    return Molecule(s)
end

create_molecule (generic function with 1 method)

In [3]:
function energy_at_bond(r)
  # define molecule geometry
  molecule = create_molecule("H",r,4)
  # build electronic hamiltonian 
  hf = molecular_orbital_hamiltonian(molecule; basis="sto-3g")
  hamiltonian = hf.hamiltonian
  hartree_fock_state = hf.hartree_fock_state
  hartree_fock_energy = hf.hartree_fock_energy
  # hilbert space
  s = siteinds("Electron", 4; conserve_qns=true)
  H = MPO(hamiltonian, s)
  # initialize MPS to HF state
  ψhf = MPS(s, hartree_fock_state)
  # run dmrg
  dmrg_kwargs = (;
    nsweeps=10,
    maxdim=[10,20,30,40,50,100],
    cutoff=1e-8,
    noise=[1e-6, 1e-7, 1e-8, 0.0],
  )
  dmrg_energy, _ = dmrg(H, ψhf; nsweeps=10, outputlevel=0)
  return hartree_fock_energy, dmrg_energy
end


energy_at_bond (generic function with 1 method)

In [4]:
# bond distances
r⃗ = 0.7:0.1:1
energies = []
for r in r⃗
  push!(energies, (r,energy_at_bond(r)))
end

converged SCF energy = -2.06919742280387
RHF Energy (Ha): -2.069197422803868
converged SCF energy = -2.12138675587022
RHF Energy (Ha): -2.121386755870221
converged SCF energy = -2.12425973897278
RHF Energy (Ha): -2.124259738972777
converged SCF energy = -2.09854593699772
RHF Energy (Ha): -2.0985459369977173


In [10]:
energies[:][2]

(0.8, (-2.121386755870221, -2.1487401180290164))

In [ ]:
function min_energy(energy)
    dm_energy = []
    for j in energy
        push!(dm_energy, j[2][2])
    end
    min = minimum(dm_energy)
    for j in energy
        if j[2][2] == min
            return j
        end
    end
end